<a href="https://colab.research.google.com/github/sakuna47/Intelligent-Resume-Screening-System/blob/Code/Intelligent_Resume_Screening_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Path to main folder
main_path = '/content/drive/MyDrive/Intelligent Resume Screening System/'
data_path = main_path + 'data/data/'
resume_path = main_path + 'Resume/Resume.csv'


Mounted at /content/drive
